In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import pathlib

In [ ]:
def augment_data(image):
    image = tf.image.resize_with_crop_or_pad(image, 180, 180) # Add 6 pixels of padding
    image = tf.image.random_crop(image, size=[150, 150, 3]) # Random crop back to 28x28
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness

    return image

In [ ]:
directory = "D:/dataset/flower_photos"

#or:

# directory = tf.keras.utils.get_file(
#     'flower_photos',
#     'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
#     untar=True)

In [ ]:
train_dir = pathlib.Path(directory)

In [ ]:
CLASS_NAMES = np.array([item.name for item in train_dir.glob('*') if item.name != "LICENSE.txt"])

In [ ]:
CLASS_NAMES

In [ ]:
full_dataset = tf.data.Dataset.list_files(str(train_dir/'*/*'))

In [ ]:
validation_split = 0.2

In [ ]:
DATASET_SIZE = len(list(full_dataset))
print("Dataset size: ", DATASET_SIZE)
train_size = int((1-validation_split) * DATASET_SIZE)
print("train size: ", train_size)
train_dataset = full_dataset.take(train_size)
validation_dataset = full_dataset.skip(train_size)

In [ ]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  return parts[-2] == CLASS_NAMES

In [ ]:
get_label("D:\\dataset\\flower_photos\\daisy\\5547758_eea9edfd54_n.jpg").numpy()

In [ ]:
def load_img(image_path):
    img = tf.io.read_file(image_path)
    
    # https://stackoverflow.com/questions/44942729/tensorflowvalueerror-images-contains-no-shape
    img = tf.image.decode_image(img, 3, expand_animations=False)
    
    img = tf.cast(img, tf.float32)
    return img

In [ ]:
# normalizing the images to [-1, 1]
def normalize(image):
    image = (image / 127.5) - 1
    return image

In [ ]:
def resize(image,height, width):
    image = tf.image.resize(image, (height, width),
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    #image = tf.image.resize_with_crop_or_pad(image, height, width)
    return image

In [ ]:
def load_image_with_label(image_path):
    label = get_label(image_path)
    img = load_img(image_path)
    return img, label

In [ ]:
def load_image_train(image_file):
    image, label = load_image_with_label(image_file)
    image = augment_data(image)
    image = normalize(image)
    
    return image, label

In [ ]:
def load_image_test(image_file):
    image, label = load_image_with_label(image_file)
    image = resize(image, 150, 150)
    image = normalize(image)

    return image, label

In [ ]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

In [ ]:
train_dataset = train_dataset.map(load_image_train)
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE)

In [ ]:
validation_dataset = validation_dataset.map(load_image_test)
validation_dataset = validation_dataset.batch(BATCH_SIZE)

In [ ]:
base_model = tf.keras.applications.VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [ ]:
base_model.trainable = False

In [ ]:
n_class =len(CLASS_NAMES)

In [ ]:
flatten_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(100, activation='relu')
dropout_layer = tf.keras.layers.Dropout(0.5)
prediction_layer = tf.keras.layers.Dense(n_class, activation='softmax')

In [ ]:
model = tf.keras.Sequential([
  base_model,
  flatten_layer,
  dense_layer,
  dropout_layer,
  prediction_layer
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.01),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset,
                    epochs=100,
                    validation_data=validation_dataset)